In [1]:
import pandas as pd
import numpy as np
import re
from rapidfuzz import process, fuzz
from unidecode import unidecode
from tqdm import tqdm
from pathlib import Path

In [2]:
# === Caminhos ===
RESULTADOS_DIR = Path("Resultados/")
RESULTADOS_DIR.mkdir(exist_ok=True)

# === Parâmetros ===
FUZZY_THRESHOLD_STRICT = 95
FUZZY_THRESHOLD_LOOSE = 75

# === Funções auxiliares ===
def limpar_titulo(titulo: str) -> str:
    """Remove acentos, pontuação e informações extras do título Netflix."""
    if not isinstance(titulo, str):
        return ""
    t = titulo.lower().strip()
    t = unidecode(t)
    # remove partes como ": season 3", ": part 2", "– episode"
    t = re.sub(r":\s*(season|seasons|part|episode|ep|chapter|capitulo)\s*\d+", "", t)
    # remove conteúdo entre parênteses e colchetes
    t = re.sub(r"[\(\[].*?[\)\]]", "", t)
    # remove pontuação e espaços duplos
    t = re.sub(r"[^a-z0-9\s]", "", t)
    t = re.sub(r"\s+", " ", t)
    return t.strip()

In [3]:
def score_com_ano(titulo, imdb_titulos, imdb_anos, ano_netflix):
    """Retorna o melhor match considerando também a proximidade de ano."""
    if not isinstance(titulo, str) or not titulo:
        return None, 0, None

    match, score, _ = process.extractOne(titulo, imdb_titulos, scorer=fuzz.token_sort_ratio)
    if match is None:
        return None, 0, None

    ano_imdb = imdb_anos.get(match, None)
    if ano_imdb and ano_netflix and abs(ano_netflix - ano_imdb) <= 1:
        score += 5  # bônus de ano próximo

    return match, min(score, 100), ano_imdb

In [4]:
# === Função principal ===
def unir_netflix_imdb():
    print("🎞️ Carregando dados tratados...\n")
    netflix = pd.read_parquet(RESULTADOS_DIR / "Netflix_Historico_Tratado.parquet", engine="fastparquet")
    imdb = pd.read_parquet(RESULTADOS_DIR / "IMDB_Limpo.parquet", engine="fastparquet")

    print(f"📂 Netflix: {len(netflix):,} títulos únicos")
    print(f"🎬 IMDB: {len(imdb):,} registros antes do filtro\n")

    # === 1️⃣ Pré-processamento ===
    netflix["titulo_limpo"] = netflix["Title"].apply(limpar_titulo)
    imdb["titulo_limpo"] = imdb["primaryTitle"].apply(limpar_titulo)

    imdb_unico = (
        imdb.sort_values("numVotes", ascending=False)
            .drop_duplicates(subset=["titulo_limpo"], keep="first")
            .reset_index(drop=True)
    )
    imdb_titulos = imdb_unico["titulo_limpo"].tolist()
    imdb_anos = dict(zip(imdb_unico["titulo_limpo"], imdb_unico["startYear"]))

    print(f"🎯 {len(imdb_unico):,} títulos únicos após normalização.\n")

    # === 2️⃣ Primeira rodada (strict matching) ===
    print("🔍 Rodada 1: Matching rigoroso (score ≥ 95)...")
    matches = []
    for _, row in tqdm(netflix.iterrows(), total=len(netflix)):
        match, score, ano_imdb = score_com_ano(row["titulo_limpo"], imdb_titulos, imdb_anos, None)
        if score >= FUZZY_THRESHOLD_STRICT:
            matches.append((row["Title"], match, score, "perfeito"))
        elif score >= FUZZY_THRESHOLD_LOOSE:
            matches.append((row["Title"], match, score, "parcial"))
        else:
            matches.append((row["Title"], None, score, "falhou"))

    matches_df = pd.DataFrame(matches, columns=["netflix_title", "imdb_title", "match_score", "match_tipo"])

    # === 3️⃣ Junta com IMDB ===
    df_final = matches_df.merge(imdb_unico, left_on="imdb_title", right_on="titulo_limpo", how="left")
    df_final["assistido"] = 1

    # === 4️⃣ Estatísticas ===
    total = len(df_final)
    perfeitos = (df_final["match_tipo"] == "perfeito").sum()
    parciais = (df_final["match_tipo"] == "parcial").sum()
    falhados = (df_final["match_tipo"] == "falhou").sum()

    print("\n📊 Qualidade do matching:")
    print(f"✅ Perfeitos: {perfeitos:,} ({perfeitos/total:.1%})")
    print(f"👍 Parciais: {parciais:,} ({parciais/total:.1%})")
    print(f"❌ Falharam: {falhados:,} ({falhados/total:.1%})")

    # === 5️⃣ Salvar resultado principal ===
    df_final.to_parquet(RESULTADOS_DIR / "Netflix_IMDB_Matched.parquet", index=False, engine="fastparquet")

    # === 6️⃣ Exportação para validação manual ===
    colunas_validacao = [
        "netflix_title", "imdb_title", "match_score", "match_tipo",
        "titleType", "startYear", "runtimeMinutes", "genres",
        "averageRating", "numVotes"
    ]

    df_validacao = df_final[colunas_validacao].sort_values(by="match_score", ascending=False)
    caminho_csv = RESULTADOS_DIR / "Netflix_IMDB_Matched_Validacao.csv"

    df_validacao.to_csv(
        caminho_csv,
        sep=";", decimal=",", index=False, encoding="utf-8-sig"
    )

    print("\n💾 Arquivos salvos com sucesso:")
    print(f" - Parquet: {RESULTADOS_DIR / 'Netflix_IMDB_Matched.parquet'}")
    print(f" - CSV (validação manual): {caminho_csv}")

    return df_final


# === Execução ===
df_matched = unir_netflix_imdb()
df_matched.head(10)

🎞️ Carregando dados tratados...

📂 Netflix: 688 títulos únicos
🎬 IMDB: 1,170,931 registros antes do filtro

🎯 982,368 títulos únicos após normalização.

🔍 Rodada 1: Matching rigoroso (score ≥ 95)...


100%|██████████| 688/688 [00:45<00:00, 15.26it/s]



📊 Qualidade do matching:
✅ Perfeitos: 631 (91.7%)
👍 Parciais: 41 (6.0%)
❌ Falharam: 16 (2.3%)

💾 Arquivos salvos com sucesso:
 - Parquet: Resultados\Netflix_IMDB_Matched.parquet
 - CSV (validação manual): Resultados\Netflix_IMDB_Matched_Validacao.csv


,netflix_title,imdb_title,match_score,match_tipo,tconst,titleType,primaryTitle,originalTitle,startYear,runtimeMinutes,genres,averageRating,numVotes,titulo_limpo,assistido
0,doom at your service,doom at your service,100.00000,perfeito,tt13669128,tvSeries,Doom at Your Service,Eoneu Nal Uri Jib Hyeongwaeuro Myeolmangyi Deu...,2021.0,65.0,"Drama,Fantasy,Romance",7.8,10596.0,doom at your service,1
1,brooklyn ninenine,brooklyn ninenine,100.00000,perfeito,tt2467372,tvSeries,Brooklyn Nine-Nine,Brooklyn Nine-Nine,2013.0,22.0,"Comedy,Crime",8.4,403619.0,brooklyn ninenine,1
2,love next door,love next door,100.00000,perfeito,tt30446769,tvSeries,Love Next Door,Eomma Chingu Adeul,2024.0,60.0,"Comedy,Romance",7.6,9330.0,love next door,1
3,romantics anonymous,romantics anonymous,100.00000,perfeito,tt1565958,movie,Romantics Anonymous,Les émotifs anonymes,2010.0,80.0,"Comedy,Romance",6.9,12410.0,romantics anonymous,1
4,beyond the bar,beyond the bar,100.00000,perfeito,tt37660730,tvSeries,Beyond the Bar,Beyond the Bar,2025.0,60.0,Drama,7.9,3752.0,beyond the bar,1
5,because this is my first life,because this is my first life,100.00000,perfeito,tt7278588,tvSeries,Because This Is My First Life,Ibeon Saengeun Cheoeumira,2017.0,70.0,"Comedy,Drama,Romance",8.0,10079.0,because this is my first life,1
6,our beloved summer,our beloved summer,100.00000,perfeito,tt15026724,tvSeries,Our Beloved Summer,Geu hae urineun,2021.0,70.0,"Drama,Romance",8.2,13620.0,our beloved summer,1
7,fight for my way,fight for my way,100.00000,perfeito,tt6824234,tvSeries,Fight for My Way,Ssam maiwei,2017.0,70.0,"Comedy,Romance",8.1,11790.0,fight for my way,1
8,alice in borderland,alice in borderland,100.00000,perfeito,tt10795658,tvSeries,Alice in Borderland,Imawa no Kuni no Arisu,2020.0,50.0,"Action,Drama,Horror",7.8,135325.0,alice in borderland,1
9,weightlifting fairy kim bok joo,weightlifting fairy kim bokjoo,88.52459,parcial,tt6157148,tvSeries,Weightlifting Fairy Kim Bok-Joo,Yeokdoyojeong Gim Bokju,2016.0,60.0,"Comedy,Drama,Romance",8.3,15896.0,weightlifting fairy kim bokjoo,1
